In [1]:
import time
import pickle
import pandas as pd
import numpy as np
import math

# internal tools
from dsgtools.reporting import make_format
from dsgtools.reporting import TableWriter
from dsgtools.reporting import freq
from dsgtools.reporting import bivariate

In [2]:
header = pd.read_adls('Analytics/Personal Folders/liuwei01/2023/account management pull (0302 Kristin)/sbfe_management_extract_file_jan2022_20230302', 
                      reader = pd.read_csv, dtype = str, encoding='iso-8859-1')
print(header.shape)

(4753095, 36)


In [3]:
freq(header["cycle_end_date"])

,Count,Pct,Cuml Count,Cuml Pct
cycle_end_date,,,,
20220101,50990.0,0.010728,50990.0,0.010728
20220102,48551.0,0.010215,99541.0,0.020942
20220103,34327.0,0.007222,133868.0,0.028164
20220104,36255.0,0.007628,170123.0,0.035792
20220105,34746.0,0.007310,204869.0,0.043102
20220106,64716.0,0.013616,269585.0,0.056718
20220107,140639.0,0.029589,410224.0,0.086307
20220108,5881.0,0.001237,416105.0,0.087544
20220109,19685.0,0.004142,435790.0,0.091686


In [4]:
header["unique_identifier"] = header["sbfe_contributor_number"] + "_" + header["contract_account_number"] + "_" + header["account_type_reported"]
header = header.rename(columns={"business_name": "BusinessName", 
                            "business_street_address": "BusinessStreetAddress", 
                            "business_city": "BusinessCity", 
                            "business_state":"BusinessState", 
                            "business_zip_code":"BusinessZipcode", 
                            "business_phone": "BusinessPhone", 
                            "business_fein": "BusinessTIN", 
                            })

In [5]:
header["InsufficientBusiness"] = np.where(header["BusinessName"].isnull() |
                            ((header["BusinessStreetAddress"].isnull() | header["BusinessCity"].isnull() | header["BusinessState"].isnull()) &
                            (header["BusinessStreetAddress"].isnull() | header["BusinessZipcode"].isnull())), 1, 0)

header["pop_owner_first"]       = np.where(header["owner_first_name"].notnull(), 1, 0)   
header["pop_owner_last"]        = np.where(header["owner_last_name"].notnull(), 1, 0)        
header["pop_owner_ssn"]         = np.where(header["owner_ssn"].notnull(), 1, 0) 
header["pop_owner_phone"]       = np.where(header["owner_phone"].notnull(), 1, 0) 
header["pop_guarantor_first"]   = np.where(header["guarantor_first_name"].notnull(), 1, 0)     
header["pop_guarantor_last"]    = np.where(header["guarantor_last_name"].notnull(), 1, 0)     
header["pop_guarantor_ssn"]     = np.where(header["guarantor_ssn"].notnull(), 1, 0) 
header["pop_guarantor_phone"]   = np.where(header["guarantor_phone"].notnull(), 1, 0)     

header["pop_owner_addr"] = np.where(header["owner_street_address"].notnull() &
                                    ((header["owner_city"].notnull() & header["owner_state"].notnull()) | header["owner_zip_code"].notnull()), 1, 0)
header["pop_guarantor_addr"] = np.where(header["guarantor_street_address"].notnull() &
                                    ((header["guarantor_city"].notnull() & header["guarantor_state"].notnull()) | header["guarantor_zip_code"].notnull()), 1, 0)

header["owner_flag"] = np.where((header["pop_owner_first"] ==1) & (header["pop_owner_last"] ==1) & 
                        ((header["pop_owner_addr"] ==1) | (header["pop_owner_ssn"] ==1)), 1, 0)

header["guarantor_flag"] = np.where((header["pop_guarantor_first"] ==1) & (header["pop_guarantor_last"] ==1) & 
                        ((header["pop_guarantor_addr"] ==1) | (header["pop_guarantor_ssn"] ==1)), 1, 0)

header["RepFirstName"] = np.where(header["owner_flag"] ==1, header["owner_first_name"], np.where(header["guarantor_flag"] ==1, header["guarantor_first_name"], ""))
header["RepLastName"] = np.where(header["owner_flag"] ==1, header["owner_last_name"], np.where(header["guarantor_flag"] ==1, header["guarantor_last_name"], ""))
header["RepStreetAddress"] = np.where(header["owner_flag"] ==1, header["owner_street_address"], 
                            np.where(header["guarantor_flag"] ==1, header["guarantor_street_address"], np.NaN))
header["RepCity"] = np.where(header["owner_flag"] ==1, header["owner_city"], 
                            np.where(header["guarantor_flag"] ==1, header["guarantor_city"], np.NaN))
header["RepState"] = np.where(header["owner_flag"] ==1, header["owner_state"], 
                            np.where(header["guarantor_flag"] ==1, header["guarantor_state"], np.NaN))
header["RepZipcode"] = np.where(header["owner_flag"] ==1, header["owner_zip_code"], 
                            np.where(header["guarantor_flag"] ==1, header["guarantor_zip_code"], np.NaN))
header["RepPhone"] = np.where(header["owner_flag"] ==1, header["owner_phone"], 
                            np.where(header["guarantor_flag"] ==1, header["guarantor_phone"], np.NaN))
header["RepSSN"] = np.where(header["owner_flag"] ==1,  header["owner_ssn"].str.zfill(9), 
                            np.where(header["guarantor_flag"] ==1, header["guarantor_ssn"].str.zfill(9), np.NaN))
header["RepLexid"] = np.where(header["owner_flag"] ==1, header["owner_lexid"], 
                            np.where(header["guarantor_flag"] ==1, header["guarantor_lexid"], np.NaN))

header["InsufficientRep"] = np.where(header["RepFirstName"].isnull() | header["RepLastName"].isnull() | 
                                    ((header["RepStreetAddress"].isnull() | header["RepCity"].isnull() | header["RepState"].isnull()) & 
                                    (header["RepStreetAddress"].isnull() | header["RepZipcode"].isnull()) 
                                    & (header["RepSSN"].isnull())), 1, 0)

In [6]:
keep = ["unique_identifier", 'cycle_end_date', 'InsufficientBusiness', 'InsufficientRep', 
        'ultid', 'orgid', 'seleid', 'proxid','powid', 'RepLexid', 
       'BusinessName', 'BusinessStreetAddress', 'BusinessCity', 'BusinessState', 'BusinessZipcode', 'BusinessPhone', 'BusinessTIN',
       'RepFirstName', 'RepLastName', 'RepStreetAddress', 'RepCity', 'RepState', 'RepZipcode', 'RepPhone', 'RepSSN', ]

In [7]:
header.isnull().sum()

sbfe_contributor_number              0
contract_account_number              0
account_type_reported                0
date_account_opened                 83
cycle_end_date                       0
multiple_account_open_dates          0
ultid                                0
orgid                                0
seleid                               0
proxid                               0
powid                                0
BusinessName                       874
BusinessStreetAddress              844
BusinessCity                       842
BusinessState                      842
BusinessZipcode                   1090
BusinessPhone                  2062538
BusinessTIN                    2586820
owner_lexid                          0
owner_first_name               4301349
owner_last_name                4301351
owner_street_address           4309729
owner_city                     4309699
owner_state                    4309699
owner_zip_code                 4309703
owner_phone              

In [8]:
header = header[keep]
print(header.shape)

(4753095, 25)


In [9]:
freq(header.InsufficientRep)

,Count,Pct,Cuml Count,Cuml Pct
InsufficientRep,,,,
0,1437514.0,0.302437,1437514.0,0.302437
1,3315581.0,0.697563,4753095.0,1.000000


In [10]:
header_dedup = header.sort_values(by=["seleid", 'InsufficientRep'], ascending = False).drop_duplicates('seleid', keep='first', ignore_index = True)
print(header_dedup.shape)

(3554530, 25)


In [11]:
freq(header_dedup.InsufficientRep)

,Count,Pct,Cuml Count,Cuml Pct
InsufficientRep,,,,
0,1104413.0,0.310706,1104413.0,0.310706
1,2450117.0,0.689294,3554530.0,1.000000


In [12]:
header_dedup["ArchiveDate"] = "202201"

In [14]:
header_dedup.to_adls('Analytics/Personal Folders/liuwei01/2023/account management pull (0302 Kristin)/input_data_202201.parquet', 
                     format = '.parquet', overwrite = True)

In [3]:
input_df = pd.read_adls('Analytics/Personal Folders/liuwei01/2023/account management pull (0302 Kristin)/input_data_202201.parquet', 
                      reader = pd.read_parquet)
print(input_df.shape)

(3554530, 26)


In [4]:
pd.DataFrame(input_df.isnull().sum())

,0
unique_identifier,0
cycle_end_date,0
InsufficientBusiness,0
InsufficientRep,0
ultid,0
orgid,0
seleid,0
proxid,0
powid,0
RepLexid,2450117


In [5]:
freq(input_df.InsufficientBusiness)

,Count,Pct,Cuml Count,Cuml Pct
InsufficientBusiness,,,,
0,3554523.0,0.999998,3554523.0,0.999998
1,7.0,0.000002,3554530.0,1.000000


In [6]:
freq(input_df.InsufficientRep)

,Count,Pct,Cuml Count,Cuml Pct
InsufficientRep,,,,
0,1104413.0,0.310706,1104413.0,0.310706
1,2450117.0,0.689294,3554530.0,1.000000
